In [1]:
!wget https://lodmedia.hb.bizmrg.com/case_files/853908/test_dataset_test.zip
!unzip -q test_dataset_test.zip

--2022-10-19 15:15:27--  https://lodmedia.hb.bizmrg.com/case_files/853908/test_dataset_test.zip
Распознаётся lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)… 217.69.142.100
Подключение к lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)|217.69.142.100|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 503 Service Unavailable
2022-10-19 15:15:27 ОШИБКА 503: Service Unavailable.



In [2]:
from PIL import Image
import torch
import os
import pandas as pd

from transformers import OwlViTProcessor, OwlViTForObjectDetection

In [3]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [4]:
processor = OwlViTProcessor.from_pretrained("google/owlvit-base-patch32")
model = OwlViTForObjectDetection.from_pretrained("google/owlvit-base-patch32")

In [5]:
texts = [["speed sign"]]
score_threshold = 0.005
max_size_threshold = 100
min_size_threshold = 15

In [6]:
data_type = "test"
data = pd.read_csv(f"{data_type}.csv")

In [7]:
for r in data.iterrows():
    if os.path.exists(f"{data_type}/{r[1].id}"):
        continue
    
    image = Image.open(f"{data_type}/{r[1].img}")
    
    inputs = processor(text=texts, images=image, return_tensors="pt")
    outputs = model(**inputs)
    
    target_sizes = torch.Tensor([image.size[::-1]])
    results = processor.post_process(outputs=outputs, target_sizes=target_sizes)

    text = texts[0]
    boxes, scores, labels = results[0]["boxes"], results[0]["scores"], results[0]["labels"]

    i = 0
    for box, score, label in zip(boxes, scores, labels):
        box = [round(i, 2) for i in box.tolist()]
        if score > score_threshold and \
            min_size_threshold < box[2] - box[0] < max_size_threshold and \
            min_size_threshold < box[3] - box[1] < max_size_threshold:
                os.makedirs(f"{data_type}/{r[1].id}", exist_ok=True)
                image.crop(box).save(f"{data_type}/{r[1].id}/{i}.jpg")
                i += 1